# Simulation

CoHDL is currently not able to simulate synthesizable contexts. Instead [cohdl_sim]() can be used. This Python module uses [cocotb]() to define testbenches for the generated output of CoHDL.

The advantage of this approach is, that the designs are tested using established tools.

The following simple example defines tests for an entity that adds two numbers. For more details checkout the cohdl_sim [repo]().

In [ ]:
from cohdl import Entity, Port, Unsigned
from cohdl import std

class MyDesign(Entity):
    inp_a = Port.input(Unsigned[8])
    inp_b = Port.input(Unsigned[8])

    out_sum = Port.output(Unsigned[8])

    def architecture(self):
        @std.concurrent
        def interleave_logic():
            self.out_sum <<= self.inp_a + self.inp_b

cohdl_sim provides a single class `Simulator`. It wraps a CoHDL entity, translates it to VHDL and starts a cocotb test session with that code. Additionally it abstracts cocotb functions so testbenches look like synthesizable CoHDL code.

In [ ]:
from cohdl_sim import Simulator

sim = Simulator(MyDesign)

@sim.test
async def testbench_1(entity: MyDesign):
    entity.inp_a <<= 1
    entity.inp_b <<= 2

    # run simulator to update output
    await sim.delta_step()

    assert entity.out_sum == 3

@sim.test
async def testbench_2(entity: MyDesign):
    for a in range(10):
        for b in range(10):
            entity.inp_a <<= a
            entity.inp_b <<= b

            await sim.delta_step()

            assert entity.out_sum == a+b